In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
from pprint import pprint 
from proj_module import get_country, convert_milliseconds_to_year_month_day

In [2]:


# create rectangular bins
bin_size = 10 # degree increments
start_lat = -90
end_lat = 90
start_long = -180
end_long = 180

min_lats = np.arange(start_lat, end_lat, bin_size)
max_lats = np.arange(start_lat + bin_size, end_lat + bin_size, bin_size)
min_longs = np.arange(start_long, end_long, bin_size)
max_longs = np.arange(start_long + bin_size, end_long + bin_size, bin_size)

# create rectangular limits array with subarrays with elements [minimum_latitude, maximum_latitude, minimum_longitude, maximum_longitude]
bins = {}
for i in range(len(min_lats)):
    for j in range(len(min_longs)):
        bins[f"{i}_{j}"] = {
            "min_lat": min_lats[i],
            "max_lat": max_lats[i],
            "min_lon": min_longs[j],
            "max_lon": max_longs[j]
        }


pprint(bins)


{'0_0': {'max_lat': -80, 'max_lon': -170, 'min_lat': -90, 'min_lon': -180},
 '0_1': {'max_lat': -80, 'max_lon': -160, 'min_lat': -90, 'min_lon': -170},
 '0_10': {'max_lat': -80, 'max_lon': -70, 'min_lat': -90, 'min_lon': -80},
 '0_11': {'max_lat': -80, 'max_lon': -60, 'min_lat': -90, 'min_lon': -70},
 '0_12': {'max_lat': -80, 'max_lon': -50, 'min_lat': -90, 'min_lon': -60},
 '0_13': {'max_lat': -80, 'max_lon': -40, 'min_lat': -90, 'min_lon': -50},
 '0_14': {'max_lat': -80, 'max_lon': -30, 'min_lat': -90, 'min_lon': -40},
 '0_15': {'max_lat': -80, 'max_lon': -20, 'min_lat': -90, 'min_lon': -30},
 '0_16': {'max_lat': -80, 'max_lon': -10, 'min_lat': -90, 'min_lon': -20},
 '0_17': {'max_lat': -80, 'max_lon': 0, 'min_lat': -90, 'min_lon': -10},
 '0_18': {'max_lat': -80, 'max_lon': 10, 'min_lat': -90, 'min_lon': 0},
 '0_19': {'max_lat': -80, 'max_lon': 20, 'min_lat': -90, 'min_lon': 10},
 '0_2': {'max_lat': -80, 'max_lon': -150, 'min_lat': -90, 'min_lon': -160},
 '0_20': {'max_lat': -80, 'ma

In [13]:
# create time window dictionary

start_year = 2010
end_year = 2023

num_days_month = {
    "01": 31,
    "03": 31,
    "04": 30,
    "05": 31,
    "06": 30,
    "07": 31,
    "08": 31,
    "09": 30,
    "10": 31,
    "11": 30,
    "12": 31
}

time_windows = {}

counter = 1
for year in range(start_year, end_year+1):
    for m in range(1, 13, 1):
        month = f"{m:02}"
        
        if month == "02":
            if year % 4 == 0:
                end_day = "29"
            else:
                end_day = "28"
        else:
            end_day = num_days_month[month]
        
        start_day = "01"
        time_windows[counter] = {
            "start_date": f"{year}-{month}-{start_day}",
            "end_date": f"{year}-{month}-{end_day}"
            }
        # print(f"{year}-{month}-{end_day}")
        counter += 1

pprint(time_windows)
        
    

{1: {'end_date': '2010-01-31', 'start_date': '2010-01-01'},
 2: {'end_date': '2010-02-28', 'start_date': '2010-02-01'},
 3: {'end_date': '2010-03-31', 'start_date': '2010-03-01'},
 4: {'end_date': '2010-04-30', 'start_date': '2010-04-01'},
 5: {'end_date': '2010-05-31', 'start_date': '2010-05-01'},
 6: {'end_date': '2010-06-30', 'start_date': '2010-06-01'},
 7: {'end_date': '2010-07-31', 'start_date': '2010-07-01'},
 8: {'end_date': '2010-08-31', 'start_date': '2010-08-01'},
 9: {'end_date': '2010-09-30', 'start_date': '2010-09-01'},
 10: {'end_date': '2010-10-31', 'start_date': '2010-10-01'},
 11: {'end_date': '2010-11-30', 'start_date': '2010-11-01'},
 12: {'end_date': '2010-12-31', 'start_date': '2010-12-01'},
 13: {'end_date': '2011-01-31', 'start_date': '2011-01-01'},
 14: {'end_date': '2011-02-28', 'start_date': '2011-02-01'},
 15: {'end_date': '2011-03-31', 'start_date': '2011-03-01'},
 16: {'end_date': '2011-04-30', 'start_date': '2011-04-01'},
 17: {'end_date': '2011-05-31', '

In [20]:
base_url = "https://earthquake.usgs.gov/fdsnws/event/1/query?"

# filters for query
min_magnitude = 5
max_depth = 100          # maximum depth of earthquake
num_results = 3000
order_by = 'magnitude'



# setup base query
query_params = {
    "format":       "geojson",
    # "starttime":     "20180101",     # Beginning of 2023 ** testing is 1 month
    # "endtime":      "20231231",
    "minmagnitude": min_magnitude,
    "maxdepth":     max_depth,
    "limit":        num_results,
    "orderby":      order_by

}

for date_index, date_range in time_windows.items():
    print(f"Starting data retrieval for {time_windows[date_index]}")
    query_params["starttime"] = date_range["start_date"]
    query_params["endtime"] = date_range["end_date"]

    # earthquake data storage dictionary
    earthquakes_data = {
        "bin_id": [],
        "bin_centre_lat": [],
        "bin_centre_lon": [],
        "quake_lat": [],
        "quake_lon": [],
        "place": [],
        "country": [],
        "magnitude": [],
        "depth": [],
        "utm": [],
        "date": []
        
    }

    for bin_index, bounds in bins.items():
        query_params["minlatitude"] = bounds["min_lat"]
        query_params["maxlatitude"] = bounds["max_lat"]
        query_params["minlongitude"] = bounds["min_lon"]
        query_params["maxlongitude"] = bounds["max_lon"]

        response = requests.get(url=base_url, params=query_params)
        
        features = response.json()["features"] # contains all the earthquake data
        metadata = response.json()["metadata"]  

        n_earthquakes = metadata["count"]

        
        had_earthquake = n_earthquakes > 0

        if had_earthquake:
            for quake in features:
                geometry = quake["geometry"]["coordinates"]
                lon = geometry[0]
                lat = geometry[1]
                depth = geometry[2]
                eq_properties = quake["properties"]
                earthquakes_data["quake_lat"].append(lat)
                earthquakes_data["quake_lon"].append(lon)
                earthquakes_data["depth"].append(depth)
                earthquakes_data["place"].append(eq_properties["place"])
                earthquakes_data["magnitude"].append(eq_properties["mag"])
                utc = eq_properties["time"]
                earthquakes_data["utm"].append(utc)
                date_readable = convert_milliseconds_to_year_month_day(utc)
                earthquakes_data["date"].append(date_readable)
        
                try:
                    earthquakes_data["country"].append(get_country(lat = lat, lon = lon))
                except Exception as e:
                    earthquakes_data["country"].append("error")
                    print(f"Found error {e} \n\n Printing 'error'")

                earthquakes_data["bin_id"].append(bin_index)
                earthquakes_data["bin_centre_lat"].append(bounds["min_lat"] + bin_size/2)
                earthquakes_data["bin_centre_lon"].append(bounds["min_lon"] + bin_size/2)

        print(f"-------------------Retreived earthquake info {date_range['start_date']}-{date_range['end_date']} from geo-bin {bin_index}-------------------")
        

    # convert data into a dataframe
    earthquakes_df = pd.DataFrame(earthquakes_data)

    # Export the City_Data into a csv
    output_filename = f"output_data/earthquakes_{date_range['start_date']}_{date_range['end_date']}.csv"
    print(f"Saving data to {output_filename}\n\n")   
    earthquakes_df.to_csv(output_filename)


Starting data retrieval for {'start_date': '2010-01-01', 'end_date': '2010-01-31'}
-------------------Retreived earthquake info 2010-01-01-2010-01-31 from geo-bin 0_0-------------------
-------------------Retreived earthquake info 2010-01-01-2010-01-31 from geo-bin 0_1-------------------
-------------------Retreived earthquake info 2010-01-01-2010-01-31 from geo-bin 0_2-------------------
-------------------Retreived earthquake info 2010-01-01-2010-01-31 from geo-bin 0_3-------------------
-------------------Retreived earthquake info 2010-01-01-2010-01-31 from geo-bin 0_4-------------------
-------------------Retreived earthquake info 2010-01-01-2010-01-31 from geo-bin 0_5-------------------
-------------------Retreived earthquake info 2010-01-01-2010-01-31 from geo-bin 0_6-------------------
-------------------Retreived earthquake info 2010-01-01-2010-01-31 from geo-bin 0_7-------------------
-------------------Retreived earthquake info 2010-01-01-2010-01-31 from geo-bin 0_8----------